In [1]:
# Example usage in a Jupyter Notebook

from src.config_loader import load_configuration_from_dict
from src.display import display_menu_info, display_bom
from src.menus import calculate_total_weight


# Define configuration paths
config_dict = {
    'menu': 'menu/sample_menu.yml',
    'products': 'products/products.yml',
    'meals': 'meals/meals.yml',
    'days': 'days/days.yml',
    'everyday': 'meals/everyday.yml',
    'debug': False
}

# Load configurations
config = load_configuration_from_dict(config_dict)

# Display menu information and check nutritional rules
display_menu_info(config.menus, config.daily_norms)

# Display Bill of Materials (BOM)
display_bom(config.menus, config.products)

# Calculate and display total weight of all menus
total_weight = calculate_total_weight(config.menus)
print(f"Общий вес: {total_weight:.0f} кг")


📗 Раскладка:
📗 Секция меню "П1 дно":
  Веса дней ['816', '832', '774', '774', '774', '848', '778'], общий вес 32650 кг
  ❗ Слишком много калорий: 3473, избыток 273
  ❗ Слишком мало углеводов: 256, нужно ещё 34
  ❗ Слишком мало углеводов: 256, нужно ещё 34
  ❗ Слишком мало углеводов: 256, нужно ещё 34
  ❗ Слишком много калорий: 3525, избыток 325
⚠ 5 предупреждений
  Меню на ночёвку 1, 6 человек: гречка-булгур
    ужин:    гречка с говядиной кронидов (Гречка 480, Пряники 300, Сухари 300, Тушенка говядина Кронидов 480)
    завтрак: булгур с курятиной кронидов (Булгур 480, Козинак 240, Лук 120, Сухари 90, Тушенка говядина Кронидов 480, Халва 240)
    перекус: перекус гаврюшкин (Батончики сникерс 300, Кешью 150, Колбаса 300, Курага 300, Сухари 300, Сыр 300, Финики 300)
  Меню на ночёвку 2, 1 человек: булгур-пшено
    ужин:    булгур с говядиной кронидов (Булгур 80, Печенье 35, Сухари 15, Тушенка говядина Кронидов 80, Халва 40)
    завтрак: пшено с курятиной кронидов (Козинак 50, Колбаса 50,

NameError: name 'blue_book' is not defined